In [ ]:
%matplotlib inline

import numpy as np
import itertools
import matplotlib
import matplotlib.pyplot as plt
from tqdm import tqdm

FM class

In [ ]:
class FuzzyMeasure:

    def __init__(self):
        """
            init function
        """
        
        self.N = 0     # number of inputs 
        self.fm = []   # nicer format to work with!
        self.g = []    # lexicographic encoding

    def get_keys_index(self):
        """
            sets up a dictionary for referencing the FM
            :return: keys to the dictionary
        """

        vls = np.arange(1, self.N + 1)
        count = 0
        Lattice = {}
        for i in range(0, self.N):
            Lattice[str(np.array([vls[i]]))] = count
            count = count + 1
        for i in range(2, self.N + 1):
            A = np.array(list(itertools.combinations(vls, i)))
            for latt_pt in A:
                Lattice[str(latt_pt)] = count
                count = count + 1
        return Lattice    
    
    def produce_lattice(self):
        """
            makes a nice little ole data structure for us to index our fuzzy measure 
        """

        index_keys = self.get_keys_index()
        Lattice = {}
        for key in index_keys.keys():
            Lattice[key] = self.g[index_keys[key]]
        return Lattice
    

Create an instance and set its values

In [ ]:
# create our integral
fuzz_measure = FuzzyMeasure()

# lets go with 4 inputs in our example
fuzz_measure.N = 4

# manually set the values
#   [ x1 x2 x3 x4 x12 x13 x14 x23 x24 x34 x123 x124 x134 x234 x1234 ]
#fuzz_measure.g = np.asarray( [ 0.11, 0.12, 0.13, 0.14, 0.21, 0.22, 0.23, 0.24, 0.25, 0.26, 0.31, 0.32, 0.33, 0.34, 1 ] ) # mean
#fuzz_measure.g = np.asarray( [ 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1 ] ) # min
#fuzz_measure.g = np.asarray( [ 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1 ] ) # max
fuzz_measure.g = np.asarray( [ 1/4, 1/4, 1/4, 1/4, 1/2, 1/2, 1/2, 1/2, 1/2, 1/2, 3/4, 3/4, 3/4, 3/4, 1 ] ) # mean
print( "lexicographic encoding FM" ) 
print( fuzz_measure.g )
fuzz_measure.fm = fuzz_measure.produce_lattice()
print( "\nfriendly encoded FM" ) 
print( fuzz_measure.fm )

# extract a value
print( "\nvalue for [1 3]" ) 
print( fuzz_measure.fm["[1 3]"] )

Draw it (do "conda install networkx"). Note, there are much better ways to visualize (see http://derektanderson.com/pubs.html)
 
  * http://derektanderson.com/pdfs/WCCI_2020_FM_Viz.pdf
  * http://derektanderson.com/pdfs/PinarFuzz2017.pdf

In [ ]:
import networkx as nx # https://networkx.github.io/documentation/stable/_downloads/networkx_reference.pdf
from itertools import chain, combinations

fig = plt.figure(figsize=(12,12))

def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)  # allows duplicate elements
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

G = nx.DiGraph()

nodesizes = np.zeros(2 ** fuzz_measure.N - 1)
index_keys = fuzz_measure.get_keys_index()
k = 0
for key in index_keys.keys():
    G.add_node(key)
    nodesizes[k] = fuzz_measure.fm[key] * 10000 # this takes the fuzzy measure weight and draws circle big/small based on its relative value
    k = k + 1
    
#print("Nodes of graph: ")
#print(G.nodes())

stuff = np.asarray( range(1,fuzz_measure.N+1) )
for i, combo in enumerate(powerset(stuff), 1):
    if( np.size(combo) > 1 ):
        # take out one at a time
        for j in range(np.size(combo)):
            listx = list(combo)
            listx.remove(combo[j])
            combosmall = tuple(listx)
            v1 = "[" + str(combosmall[0])
            for k in range(1,np.size(combosmall)):
                v1 = v1 + " " + str(combosmall[k])
            v1 = v1 + "]"
            v2 = "[" + str(combo[0])
            for k in range(1,np.size(combo)):
                v2 = v2 + " " + str(combo[k])
            v2 = v2 + "]"            
            edge = (v1,v2)
            G.add_edge(*edge)

#print("Edges: ")
#print(G.edges())

pos = nx.shell_layout(G)
nx.draw_networkx_nodes(G,
                       pos,
                       with_labels=True,
                       node_color='c',
                       edgecolors='k',
                       node_size=nodesizes,
                       alpha=0.5)
nx.draw_networkx_edges(G, 
                       pos,
                       width=1.0,
                       arrows=True,
                       arrowstyle='-|>',
                       arrowsize=15,
                       edge_color='black',
                       alpha=0.5)
node_label_dict = {u: u for u in G.nodes()}
nx.draw_networkx_labels(G, 
                        pos,
                        labels=node_label_dict,
                        font_weight='bold',
                        font_size='20',
                        font_color='blue')

plt.show() # display